# Face sample Collector

In [1]:
import cv2
import os

In [2]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Function for Extracting face from image

In [3]:
# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face

# Input name of the person

In [4]:
name=input("Enter your Name here :")

Enter your Name here :Kapil


# Creation of folder to store images

In [5]:
os.chdir("C://Users//Kapil//Desktop//Mlops Project//FaceRecognition_vgg16//Data//train//")
os.system(f"mkdir {name}")
os.chdir("C://Users//Kapil//Desktop//Mlops Project//FaceRecognition_vgg16//Data//Validation//")
os.system(f"mkdir {name}")

0

# Collecting Sample of Training Set

In [6]:
# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 70 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = f'C://Users//Kapil//Desktop//Mlops Project//FaceRecognition_vgg16//Data//train//{name}//' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (100, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (255,0,0), 3)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 70: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

# Collecting sample of testing set

In [7]:
# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 30 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = f'C://Users//Kapil//Desktop//Mlops Project//FaceRecognition_vgg16//Data//Validation//{name}//' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (100, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (255,0,0), 3)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 30: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

# Loading the VGG16 Model

In [8]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


# Let's make a function that returns our FC Head

In [9]:
def layer_adder(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

# Let's add our FC Head back onto VGG

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 4

FC_Head = layer_adder(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C://Users//Kapil//Desktop//Mlops Project//FaceRecognition_vgg16//Data//train//'
validation_data_dir = 'C://Users//Kapil//Desktop//Mlops Project//FaceRecognition//Data//Validation//'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
batch_size = 1
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 280 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [12]:
os.chdir("C://Users//Kapil//Desktop//Mlops Project//FaceRecognition//")

In [13]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face_detector.h15",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 280
nb_validation_samples = 120
epochs = 5
batch_size = 1

history = modelnew.fit_generator(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    )


Epoch 1/5
280/280 [==============================] - 132s 473ms/step - loss: 0.8607 - accuracy: 0.6393 - val_loss: 0.0012 - val_accuracy: 0.9583

Epoch 00001: val_loss improved from inf to 0.00119, saving model to face_detector.h15
Epoch 2/5
280/280 [==============================] - 132s 471ms/step - loss: 0.3343 - accuracy: 0.9000 - val_loss: 0.0076 - val_accuracy: 0.7500

Epoch 00002: val_loss did not improve from 0.00119
Epoch 3/5
280/280 [==============================] - 132s 470ms/step - loss: 0.3371 - accuracy: 0.9250 - val_loss: 4.2915e-06 - val_accuracy: 0.9083

Epoch 00003: val_loss improved from 0.00119 to 0.00000, saving model to face_detector.h15
Epoch 4/5
280/280 [==============================] - 134s 478ms/step - loss: 0.2177 - accuracy: 0.9393 - val_loss: 9.7751e-06 - val_accuracy: 0.9750

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/5
280/280 [==============================] - 132s 471ms/step - loss: 0.3222 - accuracy: 0.9393 - val_loss: 6.0437e-05 - va

In [14]:
from keras.models import load_model

classifier = load_model('face_detector.h15')

In [17]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

name_dict = {"[0]": "Kapil", 
             "[1]": "Nidhi",
             "[2]": "Pihu",
             "[3]": "vanshu",}



def draw_test(name, pred, im):
    ndict = name_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, ndict, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + path_class)
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"//"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("C://Users//Kapil//Desktop//Mlops Project//FaceRecognition//Data//Validation//")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    print(res)
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Nidhi
[1]
Class - Vanshu
[2]
Class - pihu
[3]
Class - Nidhi
[1]
Class - Kapil
[0]
Class - Nidhi
[1]
Class - Vanshu
[0]
Class - Nidhi
[1]
Class - Nidhi
[1]
Class - Nidhi
[1]
